In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import math as math
import random
from time import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from scipy.linalg import svd
import statistics
from matplotlib import colors, cm
#!pip install sci-kit spatial
#from sk.spatial.objects import Line
class SOM():
    def __init__(self, dataset, topoMap, initMethod, N, size, seed=None): # initializes our SOM
        self.dataset = dataset
        self.topoMap = topoMap
        self.initMethod = initMethod
        self.N = N
        self.size = int(size)
        self.seed = seed
        self.map_nodes = N**2

        if topoMap == 'line':
          self.map_nodes = N
        random_x = [0] * self.size
        random_y = [0] * self.size
        random_z = [0] * self.size
        random_a = [0] * self.size

        #generate datasets
        if(self.dataset == "swiss_roll"):

          #saves swissroll dataset as a csv file (write to csv)

          x, t = make_swiss_roll(n_samples=self.size)

          random_x=[(x[i,0]) for i in range(self.size)]
          random_y=[(x[i,1]) for i in range(self.size)]
          random_z=[(x[i,2]) for i in range(self.size)]

          meanx = statistics.mean(random_x)
          meany = statistics.mean(random_y)
          meanz = statistics.mean(random_z)

          for i in range(self.size):
            random_x[i] -= meanx
            random_y[i] -= meany
            random_z[i] -= meanz

          random_d = [random_x, random_y, random_z]
          np.savetxt('swiss_roll.csv', (random_x, random_y, random_z), delimiter=',',fmt='%f')

          #loads swissroll dataset from a csv file (read csv)
          data = pd.read_csv("swiss_roll.csv", header=None)

          random_d = []
          for i in range(len(data)):
            temp = []
            for j in range(len(data.columns)):
              temp.append(data.iat[i, j])
            random_d.append(temp)
          for i in range(len(random_d)):
            print(random_d[i])

        elif(self.dataset == "s_curve"):
          x, t = make_s_curve(n_samples=self.size)

          random_x=[(x[i,0]) for i in range(self.size)]
          random_y=[(x[i,1]) for i in range(self.size)]
          random_z=[(x[i,2]) for i in range(self.size)]

          meanx = statistics.mean(random_x)
          meany = statistics.mean(random_y)
          meanz = statistics.mean(random_z)

          for i in range(self.size):
            random_x[i] -= meanx
            random_y[i] -= meany
            random_z[i] -= meanz

          random_d = [random_x, random_y, random_z]
          print(random_d)

        elif(topoMap == 'hexagonal'):
          x, y = np.meshgrid(np.arange(0,self.N,1.0), np.arange(0,self.N,1.0)) #places mesh grid on data
          for i in y:
            for j in range(1,len(i),2):
              i[j] += 0.5

          self.c = np.hstack((y.flatten()[:, np.newaxis], #c is the topological distance of each node
                           x.flatten()[:, np.newaxis])) # basically makes a 20x20 grid
          #print(self.c)
          fig = plt.figure(figsize=(10,10))
          ax = plt.axes(projection=None)
          ax.scatter(x, y, cmap=plt.cm.Spectral)
        elif(topoMap == 'line'):
          x = np.arange(N)
          y = np.zeros(N)
          self.c = np.vstack((x, y)).T
        #find the center of the dataset
        center = []
        for i in random_d:
          sum = 0
          for j in i:
            sum += j
          center.append(sum/len(i))
        #print(center)

        #display the initialization
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection='3d')
        #ax.scatter(random_x, random_y, random_z, cmap=plt.cm.Spectral)

        teachers = []
        #format the x,y,z cords in one array
        random_d = np.array(random_d).T
        print(random_d)
        for i in range(len(random_d)):
          teachers.append(random_d[i])
        self.teachers = np.array(teachers)
        self.n_teacher = self.teachers.shape[0]

        if(initMethod == "random"):
          #print(self.teachers.shape)
          self.nodes = np.random.rand(self.N*self.N, self.teachers.shape[1]) #nodes is the actual point in space of the node
          #print(self.nodes)
            #adding the center of the data to every node
          for node in self.nodes:
            for i in range(len(node)):
              node[i] += center[i]


        elif(initMethod == "pca"):
          U, s, VT = svd(teachers)
          #print(VT)
          VT = np.transpose(VT)  # this is just so we dont have to rewrite code cuz our PCA is perpendicular
          #Rotation
          V = np.transpose(VT)
          V = V[0:2,:]
          #print("\nV:", V)
          Vnew = np.transpose(VT)
          Vnew = Vnew[:,0:2]
          #print("\nVnew:",Vnew)
          self.W = np.hstack((y.flatten()[:, np.newaxis], # node network
                           x.flatten()[:, np.newaxis]))
          #print(self.W)
          for i in range(0,self.map_nodes): # center the dataset
            self.W[i][0] -= (self.N-1)/2
            self.W[i][1] -= (self.N-1)/2
          sum1 = 0
          sum2 = 0
          for i in range(0,self.map_nodes):
            sum1 += self.W[i][0]
            sum2 += self.W[i][1]
          #print(sum1/self.map_nodes)
          #print(sum2/self.map_nodes)
          self.Wnew = np.dot(self.W,V) # dot product of first 2 rows of V and node network rotates the node network
          # right now we do a dot product of 400 * 2 with 2 * 3
          # try doing a dot product of 3 * 2 with 2 * 400
          # so we need to rotate the self.W and dot that with VT giving us 3 * 400
          # then rotate Wnew back 90 degrees to get 400 * 3
          self.WT = np.transpose(self.W)
          #self.Wnew = np.dot(Vnew,self.WT)
          #self.Wnew = np.transpose(self.Wnew)
          #print("\nWnew:",self.Wnew)
          #print(np.shape(self.Wnew))

          #Dilation
          self.z = np.dot(self.teachers,Vnew) # dot product of first 2 cols of V and dataset represents rotated dataset
          # this is to find the range to dilate our node network W
          #print("\nZ:",self.z)
          self.fixedz = np.dot(self.z,V)
          ax.scatter(self.fixedz[:,0], self.fixedz[:,1], self.fixedz[:,2], color='yellow')
          z1Min = min(self.z[:,0:1])[0]
          z1Max = max(self.z[:,0:1])[0]
          z2Min = min(self.z[:,1:2])[0]
          z2Max = max(self.z[:,1:2])[0]
          #print(z1Max)
          #print(z1Min)
          z1Range = (z1Max - z1Min)/(self.N)
          z2Range = (z2Max - z2Min)/(self.N)
          #print(z1Range)
          dilateMatrix = [[z1Range, 0], [0,z2Range]]
          #print("\ntransformedMatrix:", dilateMatrix)
          scaledMatrix = np.dot(self.W,dilateMatrix) # scale our rotated node network by the dilated matrix
          self.Wbrandnew = np.dot(scaledMatrix, V)
          #self.Wbrandnew = np.transpose(self.Wbrandnew)
          #print("\nWbrandnew:", self.Wbrandnew)
          #print(np.shape(self.Wbrandnew))

          #verify numerically that the PCA works
          '''
          print(max(np.dot(self.Wbrandnew, Vnew[:,0:1])))
          print(">")
          print(max(np.dot(self.fixedz, Vnew[:,0:1])))

          print(min(np.dot(self.Wbrandnew, Vnew[:,1:2])))
          print("<")
          print(min(np.dot(self.fixedz, Vnew[:,1:2])))
          '''

          #Display
          ax.scatter(self.Wbrandnew[:,0],self.Wbrandnew[:,1],self.Wbrandnew[:,2], color='red')
          #ax.set_xlim3d(-12, 15)
          #ax.set_ylim3d(-1, 23)
          #ax.set_zlim3d(-13, 16)
          ax.view_init(0,90) # used for swissroll
          #ax.view_init(90,180) # used for hyperbolic parabola
          #ax.view_init(90,270) # used for s_curve
          plt.show()

          self.nodes = self.Wbrandnew
          #print(self.nodes)
          # look at the pca from the perspective of v3 and make sure the pca covers the entire projected dataset

    def _best_matching_unit(self, teacher, k=1): # finds the BMU of a node
        #compute all norms (eculidian distance)
        norms = np.linalg.norm(self.nodes - teacher, axis=1) # takes the euclidian distance between a node and a teacher
        if(k == 1):
          bmu = np.argmin(norms) # argument with smallest distance from the teacher
          return np.unravel_index(bmu,(self.N, self.N)) # returns the i and j of where the bmu is
        else:
          #TBD: use argpartition to optimize this (make it run faster)
          arr = np.argsort(norms)[:2]
          bmu1 = np.unravel_index(arr[0],(self.N, self.N))
          bmu2 = np.unravel_index(arr[1],(self.N, self.N))
          bmus = [bmu1,bmu2]
          #print(bmus)
          return bmus

    def _neighborhood(self, t): #the "gamma" the percent of nodes that move the percent of the way to teacher
        halflife = self.n_teacher/10
        initial  = 1
        final = 0.001
        return max(initial*np.exp(-t/halflife), final) #the math part for matching/updating
        #return 0.001

    def _learning_ratio(self, t): #the "alpha" the percent of the way BMU moves to teacher
        halflife = self.n_teacher/10
        initial  = 0.5
        final = 0.001
        return max(initial*np.exp(-t/halflife), final)

    def _learning_radius(self, t, d):
        # d is distance from BMU
        s = self._neighborhood(t)
        return np.exp(-d**2/(2*s**2))

    def train(self): #training algoirthm
        index_arr = np.random.randint(len(self.teachers),size=len(self.teachers)) #first num is # of data points, second num is # of iterations
        for i in range(len(index_arr)):
            #print(self.teachers[index_arr[i]])
            bmu = self._best_matching_unit(self.teachers[index_arr[i]])
            d = np.linalg.norm(self.c - bmu, axis=1)
            L = self._learning_ratio(i)
            S = self._learning_radius(i, d)
            self.nodes += L * S[:, np.newaxis] * (self.teachers[index_arr[i]] - self.nodes)
        return self.nodes

    def train_log(self): # training but only displays plots exponentially
        power = 0
        counter = 1
        index_arr = np.random.randint(len(self.teachers),size=len(self.teachers)) #first num is # of data points, second num is # of iterations
        for i in range(len(index_arr)):
            bmu = self._best_matching_unit(self.teachers[index_arr[i]])
            #print(self.nodes)
            d = np.linalg.norm(self.c - bmu, axis=1)
            #print("D: ", d)
            L = self._learning_ratio(i)
            S = self._learning_radius(i, d)
            #print("S: ", S)
            self.nodes += L * S[:, np.newaxis] * (self.teachers[index_arr[i]] - self.nodes)
            if(i % counter == 0):
              fig = plt.figure(figsize=(10,10))
              ax = plt.axes(projection='3d')
              print(counter)
              print("BMU:", self.nodes[bmu[0]*self.N + bmu[1]])
              #self.plot_dataset(fig, ax)
              #self.plot_nodes(fig, ax)
              self.edge_map()
              plt.show()
              self.u_matrix()
              power += 1
              counter = 2 ** power
        return self.nodes

    #first num is # of data points, second num is # of iterations
        for i in range(len(indexArr)):
            bmu = self._best_matching_unit(self.teachers[indexArr[i]])
            d = np.linalg.norm(self.c - bmu, axis=1)
            L = self._learning_ratio(i)
            S = self._learning_radius(i, d)
            self.nodes += L * S[:, np.newaxis] * (self.teachers[indexArr[i]] - self.nodes)
            for index in indexList:
              if(index == i):
                print(index)
                self.edge_map()
        return self.nodes

    def u_matrix(self): # shows distances between points on the SOM || higher value means farther distance and less dense SOM, basically inverse heatmap
        uMatrixMap = np.zeros([self.N,self.N]) #2d array of 0s
        #loop through self.c (topological map)
        for i in range(0, self.map_nodes):
          adjacent = []
          for j in range(0, self.map_nodes):
            difference = abs(self.c[i] - self.c[j])
            #print(difference)
            if(self.topoMap == "square"):
              if(0 in difference and 1 in difference):
                #print(self.nodes[i])
                #print(self.nodes[j])
                distance = np.linalg.norm(self.nodes[i] - self.nodes[j], ord=2)
                adjacent.append(distance)
                #print(distance)
                # self.c[i] - self.c[j] will return a point like [0,0]
                # ex. [0,0] - [0,1] = [0,1] [0,-1] [1,0] [-1,0]
                # ex. [7,8] - [7,7] = [0,1]
                # to check for adjacency, check if the difference includes a 0 in one position and either a 1 or -1 in the other
                # once we have that, find the x,y,z coords of self.nodes[i] and self.nodes[j] and use euclidian distance formula to find distance
                # put that distance into our distance map
            elif(self.topoMap == "hexagonal"):
              if((0 in difference and 1 in difference) or (1 in difference and 0.5 in difference)):   #first part is adjacency for nodes above and below and second part is for adjacency for nodes to the left and right
                #print(difference)
                distance = np.linalg.norm(self.nodes[i] - self.nodes[j], ord=2)
                adjacent.append(distance)
            elif(self.topoMap == "line"):
              if(1 in difference):
                distance = np.linalg.norm(self.nodes[i] - self.nodes[j], ord=2)
                adjacent.append(distance)
          sum = 0
          #print(adjacent)
          for distance in adjacent:
            sum += distance
          avg = sum/len(adjacent)
          x = i // self.N
          y = i % self.N
          uMatrixMap[x][y] = avg
        #print(uMatrixMap)
        figure = plt.figure()
        if(self.topoMap == 'square' or self.topoMap == 'line'):
          plt.pcolor(uMatrixMap.T, cmap="viridis")
        elif(self.topoMap == 'hexagonal'):
          ax = plt.gca()
          ax.cla()
          ax.set_xlim(-1,self.N)
          ax.set_ylim(-1,self.N)
          ax.set_aspect('equal')

          uMatrixMap[:,:] /= np.amax(uMatrixMap) # convert the map to be between 0->1 by dividing each element by the highest value
          #print(uMatrixMap)
          viridisMap = cm.viridis(uMatrixMap[:,:])
          #print(viridisMap)
          colorMap = [] # holds the hex colors of each value in uMatrixMap
          for a in viridisMap:
            temp = []
            for b in a:
              temp.append(colors.rgb2hex(b))
            colorMap.append(temp)
          #print(colorMap)

          for xcoord in range(self.N): # creates the hexagons
            for ycoord in range(self.N):
              if(xcoord % 2 == 1):
                hex = patches.RegularPolygon((xcoord, ycoord+0.5), numVertices=6, radius=2. / 3.,
                          orientation=np.radians(30),
                          facecolor=colorMap[xcoord][ycoord], alpha=1, edgecolor='k')
                ax.add_patch(hex)
              else:
                hex = patches.RegularPolygon((xcoord, ycoord), numVertices=6, radius=2. / 3.,
                          orientation=np.radians(30),
                          facecolor=colorMap[xcoord][ycoord], alpha=1, edgecolor='k')
                ax.add_patch(hex)
        #plt.colorbar()
        plt.show()

    def edge_map(self, color=False): # shows the connection between nodes
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection='3d')
        self.plot_nodes(fig, ax, color=color)
        for i in range(0, self.map_nodes):
          adjacent = []
          for j in range(0, self.map_nodes):
            difference = abs(self.c[i] - self.c[j])
            if(self.topoMap == "square"):
              if(0 in difference and 1 in difference):
                adjacent.append(j) # appends the index of the adjacent node
            elif(self.topoMap == "hexagonal"):
              if((0 in difference and 1 in difference) or (1 in difference and 0.5 in difference)):
                adjacent.append(j) # appends the index of the adjacent node
            elif(self.topoMap == "line"):
              if(1 in difference):
                adjacent.append(j)
          for k in range(len(adjacent)):
            # takes the x,y,z value of each adjacent node and the ith node to form a pair of points
            x = np.array((self.nodes[i][0], self.nodes[adjacent[k]][0]))
            y = np.array((self.nodes[i][1], self.nodes[adjacent[k]][1]))
            z = np.array((self.nodes[i][2], self.nodes[adjacent[k]][2]))
            ax.plot(x,y,z,c='black',alpha=0.5) # plots the edges between the pair of points
        #ax.set_xlim3d([-99, -97])
        #ax.set_ylim3d([39, 43])
        #ax.set_zlim3d([0, 1])
        plt.show() # displays every node as well as the adjacent edges between each node

    def button_plot(self): # shows the "breaks" in the data
        closest = [[] for _ in range(0,self.map_nodes)]
        for i in range(0,len(self.teachers)):
          index = 0
          min_distance = 10000000
          bmu = self._best_matching_unit(self.teachers[i]) # finds the BMU for each data point
          current_distance = np.linalg.norm(self.teachers[i] - self.nodes[bmu[0]*self.N + bmu[1]], ord=2)
          if(current_distance < min_distance): # find the data point that is closest to a given node
            min_distance = current_distance
            index = bmu[0]*self.N + bmu[1]
          closest[index].append(i) # add data point to the list
        ax = plt.gca()
        ax.cla()
        if(self.topoMap == 'square'):
          for i in range(0,self.map_nodes): # creates the empty node images
            circle = plt.Circle((i//self.N, i%self.N), 0.5, color='black', fill=False)
            ax.set_xlim((-1, self.N))
            ax.set_ylim((-1, self.N))
            ax.add_patch(circle)
          for i in range(len(closest)):
            for j in range(len(closest[i])):
              random_theta = np.random.rand() * 2 * np.pi
              random_r = np.random.rand() * 0.45
              x = (random_r * np.cos(random_theta)) + i // self.N
              y = (random_r * np.sin(random_theta)) + i % self.N
              ax.plot(x,y,'o',markersize=3)
          plt.show()
        elif(self.topoMap == 'line'):
          ax.set_xlim((-1, self.N))
          ax.set_ylim((-1, 1))
          for i in range(0, self.map_nodes):
            circle = plt.Circle((i, 0), 0.5, color='black', fill=False)
            ax.add_patch(circle)
          for i in range(len(closest)):
            for j in range(len(closest[i])):
              random_theta = np.random.rand() * 2 * np.pi
              random_r = np.random.rand() * 0.45
              x = (random_r * np.cos(random_theta)) + i
              y = (random_r * np.sin(random_theta))
              ax.plot(x,y,'o',markersize=3)
          plt.show()
        elif(self.topoMap == 'hexagonal'):
          shift = True
          for i in range(0,self.map_nodes): # creates the empty node images
            if(i%self.N == 0):
              if(shift):
                shift = False
              else:
                shift = True
            if(shift):
              circle = plt.Circle((i//self.N, (i%self.N) + 0.5), 0.5, color='black', fill=False)
              ax.set_xlim((-1, self.N))
              ax.set_ylim((-1, self.N+1))
              ax.add_patch(circle)
            else:
              circle = plt.Circle((i//self.N, i%self.N), 0.5, color='black', fill=False)
              ax.set_xlim((-1, self.N))
              ax.set_ylim((-1, self.N+1))
              ax.add_patch(circle)
          shift = True
          for i in range(len(closest)):
            if(i%self.N == 0):
              if(shift):
                shift = False
              else:
                shift = True
            for j in range(len(closest[i])):
              random_theta = np.random.rand() * 2 * np.pi
              random_r = np.random.rand() * 0.45
              if(shift):
                x = (random_r * np.cos(random_theta)) + i // self.N
                y = (random_r * np.sin(random_theta)) + i % self.N + 0.5
              else:
                x = (random_r * np.cos(random_theta)) + i // self.N
                y = (random_r * np.sin(random_theta)) + i % self.N
              ax.plot(x,y,'o',markersize=3)
          plt.show()
        #print(closest)

    def plot_dataset(self, figure=plt.figure(figsize=(10,10)), axes=plt.axes(projection='3d')): # plots all of the nodes in the SOM
        x_ints=[i[0] for i in self.teachers]
        y_ints=[i[1] for i in self.teachers]
        z_ints=[i[2] for i in self.teachers]
        fig = figure
        ax = axes
        ax.scatter(x_ints,y_ints,z_ints, cmap=plt.cm.Spectral)
        #plt.show()

    def plot_nodes(self, figure=plt.figure(figsize=(10,10)), axes=plt.axes(projection='3d'), color=False): # plots all of the nodes in the SOM
        x_ints=[i[0] for i in self.nodes]
        y_ints=[i[1] for i in self.nodes]
        z_ints=[i[2] for i in self.nodes]
        fig = figure
        ax = axes
        if(color == False):
          ax.scatter(x_ints,y_ints,z_ints,c='orange')
        else:
          colors = []
          for point in self.nodes:
            colors.append([point[3], point[4], point[5]]) # i[3] is red, i[4] is green, and i[5] is blue
          ax.scatter(x_ints,y_ints,z_ints,c=colors)
        #plt.show()

    def quant_error(self, teachers): # Quantization error
        distance_sum = 0
        for i in range(0,len(teachers)):
          min_distance = 10000000
          bmu = self._best_matching_unit(teachers[i]) # finds the BMU for each data point
          current_distance = np.linalg.norm(self.teachers[i] - self.nodes[bmu[0]*self.N + bmu[1]], ord=2)
          if(current_distance < min_distance): # find the data point that is closest to a given node
            min_distance = current_distance
          distance_sum += min_distance # adds the distance of each data point that is closest to a given node
        error = distance_sum/len(teachers) # finds the average distance
        return error

    def topo_error(self, teachers): # Topographic Error
        error_sum = 0
        for i in range(0, len(teachers)):
          bmus = self._best_matching_unit(teachers[i], k=2) # finds the two closest BMUs for each data point
          difference_x = bmus[0][0] - bmus[1][0]
          difference_y = bmus[0][1] - bmus[1][1]
          if((difference_x == 0 and difference_y == -1) or (difference_x == 0 and difference_y == 1) or (difference_x == 1 and difference_y == 0) or (difference_x == -1 and difference_y == 0)): # checks if BMUs are next to each other
            error_sum += 0
          else:
            error_sum += 1
        error = error_sum/len(teachers)
        return error

    def error_entropy(self):
        closest = np.zeros(self.map_nodes)
        size = np.zeros(self.map_nodes)
        for i in range(0,len(self.teachers)):
          index = 0
          min_distance = 10000000
          bmu = self._best_matching_unit(self.teachers[i]) # finds the BMU for each data point
          current_distance = np.linalg.norm(self.teachers[i] - self.nodes[bmu[0]*self.N + bmu[1]], ord=2)
          if(current_distance < min_distance): # find the data point that is closest to a given node
            min_distance = current_distance
            index = bmu[0]*self.N + bmu[1]
          closest[index] = closest[index] + (distance(self.teachers[i], self.nodes[index])) # add data point to the list
          size[index] = size[index] + 1
        E = 0
        entropy_error = 0

        for i in range(0, len(closest)):
            E = E + closest[i]
        for i in range(0, len(closest)):
            e = closest[i] / E
            entropy_error = entropy_error + e + math.log(e, 2)
        # lower better
        return -entropy_error

    def trustworthiness(self, teachers, k): # Trustworthiness (scrapped for later when we have more manpower)
        errors = 0
        for i in range(0, len(teachers)):
          setA = [] # k nearest data points for each data point
          for j in range(0, len(teachers)):
            distance = np.linalg.norm(self.teachers[i] - self.teachers[j])
            setA.append([distance,j])
          setA.sort()
          setA = setA[1:k+1]
          norms = np.linalg.norm(self.nodes - teachers[i], axis=1)
          bmu = np.argmin(norms)
          #print(bmu)
          setB = [] # k nearest data points for the BMU
          for j in range(0, len(teachers)):
             distance = np.linalg.norm(self.nodes[bmu] - self.teachers[j])
             setB.append([distance, j])
          setB.sort()
          setB = setB[1:k+1]
          for j in range(0, len(setA)):
            print(setA[j][1], setB[j][1])
          #print(errors)
          #for j in range(0,len(setB)):
            #print(setA[j], setB[j])
          print(i)
        return errors

    def grader(self, iterationNumber): #grader function
        quantError = []
        topoError = []
        for i in range(iterationNumber): #for # of iterations
          som = SOM(dataset=self.dataset, topoMap=self.topoMap, initMethod=self.initMethod, N=self.N, size=self.size, seed=self.seed) #create/initialize ur som
          som.train() #train ur som
          som.edge_map() #visualize som with edges
          som.u_matrix()
          #som.button_plot()

          quant = som.quant_error(som.teachers)
          topo = som.topo_error(som.teachers)
          quantError.append(quant) #store quant error in list
          topoError.append(topo) #store topo error in list
          #print iteration number
          #print("Iteration: " + str(i) + ", Quantization Error: " + str(quant) + ", Topographic Error: " + str(topo))
          #print quant and topo error

        # always good to use variables so we dont have to recalculate, saves on run time
        quantMean = statistics.mean(quantError)
        quantStdev = statistics.stdev(quantError)
        quantRange = max(quantError) - min(quantError)
        topoMean = statistics.mean(topoError)
        topoStdev = statistics.stdev(topoError)
        topoRange = max(topoError) - min(topoError)

        # grouped the statistics by its error function so we can easily compare within each error function
        '''
        print("QuantError List:", quantError) #print quanterror list
        print("Quant Mean:", quantMean) #print and calculate mean
        print("Quant stdev:", quantStdev) #print and calculate std deviation
        print("Quant Range:", quantRange) #print and calculate the range

        print("\nTopoError List:", topoError) #print topoerror list
        print("Topo Mean:", topoMean) #print and calculate mean
        print("Topo stdev:", topoStdev) #print and calculate std deviation
        print("Topo Range:", topoRange) #print and calculate the range
        '''

        # put the final statistics at the end so we can easily compare between different error functions
        finalQuantErrorList = ["Quantitative Error:", ("Mean", quantMean), ("Standard Deviation", quantStdev), ("Range", quantRange)]
        finalTopoErrorList = ["Topological Error:", ("Mean", topoMean), ("Standard Deviation", topoStdev), ("Range", topoRange)]
        error_list = []
        error_list.append(finalQuantErrorList)
        error_list.append(finalTopoErrorList)
        #print(error_list)
        return error_list

    def grader_to_file(self, file_name, iterationNumber): #grader function for files
        quantError = []
        topoError = []
        figureList = []
        for i in range(iterationNumber): #for # of iterations
          som = SOM(dataset=self.dataset, initMethod=self.initMethod, N=self.N, size=self.size, seed=self.seed) #create/initialize ur som
          som.train() #train ur som
          figureList.append(som.edge_map()) #visualize som with edges
          figureList.append(som.u_matrix())
          figureList.append(som.button_plot())

          quant = som.quant_error(som.teachers)
          topo = som.topo_error(som.teachers)
          quantError.append(quant) #store quant error in list
          topoError.append(topo) #store topo error in list
          #print iteration number
          #print("Iteration: " + str(i) + ", Quantization Error: " + str(quant) + ", Topographic Error: " + str(topo))
          #print quant and topo error

        p = PdfPages(file_name)
        for fig in figureList:
            fig.savefig(p, format='pdf')
        p.close()

        # always good to use variables so we dont have to recalculate, saves on run time
        quantMean = statistics.mean(quantError)
        quantStdev = statistics.stdev(quantError)
        quantRange = max(quantError) - min(quantError)
        topoMean = statistics.mean(topoError)
        topoStdev = statistics.stdev(topoError)
        topoRange = max(topoError) - min(topoError)

        # grouped the statistics by its error function so we can easily compare within each error function
        '''
        print("QuantError List:", quantError) #print quanterror list
        print("Quant Mean:", quantMean) #print and calculate mean
        print("Quant stdev:", quantStdev) #print and calculate std deviation
        print("Quant Range:", quantRange) #print and calculate the range

        print("\nTopoError List:", topoError) #print topoerror list
        print("Topo Mean:", topoMean) #print and calculate mean
        print("Topo stdev:", topoStdev) #print and calculate std deviation
        print("Topo Range:", topoRange) #print and calculate the range
        '''

        # put the final statistics at the end so we can easily compare between different error functions
        finalQuantErrorList = ["Quantitative Error:", ("Mean", quantMean), ("Standard Deviation", quantStdev), ("Range", quantRange)]
        finalTopoErrorList = ["Topological Error:", ("Mean", topoMean), ("Standard Deviation", topoStdev), ("Range", topoRange)]
        error_list = []
        error_list.append(finalQuantErrorList)
        error_list.append(finalTopoErrorList)
        #print(error_list)

        f = open(file_name, "a")
        f.write("Hello")
        for elist in error_list:
          for element in elist:
            f.write(''.join(str(element)))
        f.close()

        return error_list

    def equation_plane(self, x1, y1, z1, x2, y2, z2, x3, y3, z3):
      a1 = x2 - x1
      b1 = y2 - y1
      c1 = z2 - z1
      a2 = x3 - x1
      b2 = y3 - y1
      c2 = z3 - z1
      a = b1 * c2 - b2 * c1
      b = a2 * c1 - a1 * c2
      c = a1 * b2 - b1 * a2
      d = (- a * x1 - b * y1 - c * z1)

      return (a, b, c, d)


    def plane_intersect(self, a, b):
      """
      a, b   4-tuples/lists
            Ax + By +Cz + D = 0
            A,B,C,D in order

      output: 2 points on line of intersection, np.arrays, shape (3,)
      """
      a_vec, b_vec = np.array(a[:3]), np.array(b[:3])

      aXb_vec = np.cross(a_vec, b_vec)

      A = np.array([a_vec, b_vec, aXb_vec])
      d = np.array([-a[3], -b[3], 0.]).reshape(3,1)

  # could add np.linalg.det(A) == 0 test to prevent linalg.solve throwing error

      p_inter = np.linalg.solve(A, d).T

      return (p_inter[0], (p_inter + aXb_vec)[0])


    def intersections(self):
      planes = []
      for i in range(self.N**2):
        store_plane_index = [i]
        store_plane_index_2 = [i]
        found_plane_1 = False
        found_plane_2 = False
        for j in range(self.N**2):
          distance = self.c[i] - self.c[j]

          # if node j is above, and diagnal right, add to the first triangle; if node j is to the right and diagnal right, add to the second triangle
          if (distance[0] == 0 and distance[1] == -1): # above
            store_plane_index.insert(1, j)
          if (distance[0] == -1 and distance[1] == -1): # diagnal right
            store_plane_index.insert(2, j)
            store_plane_index_2.insert(2, j)
          if (distance[0] == -1 and distance[1] == 0): # right
            store_plane_index_2.insert(1, j)

          #print(store_plane_index)

          if len(store_plane_index) == 3: # if there are enough nodes to make a proper plane
            store_plane = []
            for k in range(len(store_plane_index)):
              xyz = (self.nodes[store_plane_index[k]][0], self.nodes[store_plane_index[k]][1], self.nodes[store_plane_index[k]][2]) # find xyz coordinates
              #print(xyz)
              store_plane.extend(xyz) # add xyz coordinates to store_plane
            planes.append(store_plane)
            found_plane_1 = True

          if len(store_plane_index_2) == 3: # if there are enough nodes to make a proper plane
            store_plane = []
            for k in range(len(store_plane_index_2)):
              xyz = (self.nodes[store_plane_index_2[k]][0], self.nodes[store_plane_index_2[k]][1], self.nodes[store_plane_index_2[k]][2]) # find xyz coordinates
              #print(xyz)
              store_plane.extend(xyz) # add xyz coordinates to store_plane
            planes.append(store_plane)
            found_plane_2 = True

          if(found_plane_1 and found_plane_2):
            break


      print(planes) # all planes
      print(len(planes))
      for points in planes:
        p0 = [points[0], points[1], points[2]]
        p1 = [points[3], points[4], points[5]]
        p2 = [points[6], points[7], points[8]]
        x0, y0, z0 = p0
        x1, y1, z1 = p1
        x2, y2, z2 = p2

        ux, uy, uz = u = [x1-x0, y1-y0, z1-z0]
        vx, vy, vz = v = [x2-x0, y2-y0, z2-z0]

        u_cross_v = [uy*vz-uz*vy, uz*vx-ux*vz, ux*vy-uy*vx]

        point  = np.array(p0)
        normal = np.array(u_cross_v)

        d = -point.dot(normal)

        xx, yy = np.meshgrid(range(10), range(10))

        z = (-normal[0] * xx - normal[1] * yy - d) * 1. / normal[2]

        # plot the surface
        plt3d = plt.axes(projection='3d')
        plt3d.plot_surface(xx, yy, z)
        plt.show()

      intersects = []
      counter = 0
      breakcounter1 = 0
      breakcounter2 = 0
      for i in planes:
        a = som.equation_plane(*i) #finding equations of planes i and j
        print(a)

        vert_line_i = Line.from_points(i[0:3], i[3:6]) #find lines for all 3 lines making up plane i
        horiz_line_i = Line.from_points(i[0:3], i[6:9])
        diag_line_i = Line.from_points(i[3:6], i[6:9])
        list_line_i = [vert_line_i, horiz_line_i, diag_line_i]

        for j in planes:
          if i == j: # not looking at the same 2 planes
            continue

          b = som.equation_plane(*j)
          print(b)
          intersects_i = []
          intersects_j = []
          #print(j)
          vert_line_j = Line.from_points(j[0:3], j[3:6]) #find lines for all 3 lines making up plane j
          horiz_line_j = Line.from_points(j[0:3], j[6:9])
          diag_line_j = Line.from_points(j[3:6], j[6:9])
          list_line_j = [vert_line_j, horiz_line_j, diag_line_j]

          try:
            intersect_points = som.plane_intersect(a, b) #find points along the line of intersection of the planes
            #print(intersect_points)
            #intersects.append(intersect_points)

            line_of_intersection = Line.from_points(intersect_points[0],intersect_points[1])
            #print(line_of_intersection)
          except:
            #print("a")
            continue

          ixmax = max(i[::3])
          ixmin = min(i[::3])
          iymax = max(i[1::3])
          iymin = min(i[1::3])
          izmax = max(i[2::3])
          izmin = min(i[2::3])



          for k in list_line_i: #test for intersection between each line making up plane i and the line of intersection to find the points of intersection
            try:
              point_intersection = line_of_intersection.intersect_line(k)
              if(point_intersection[0] > ixmin and point_intersection[0] < ixmax and point_intersection[1] > iymin and point_intersection[1] < iymax and point_intersection[2] > izmin and point_intersection[2] < izmax):
                #print(point_intersection)
                intersects_i.append(point_intersection.tolist())
            except:
              #print("b")
              pass
          if(len(intersects_i) < 2):
            breakcounter1 += 1
            continue

          jxmax = max(j[::3])
          jxmin = min(j[::3])
          jymax = max(j[1::3])
          jymin = min(j[1::3])
          jzmax = max(j[2::3])
          jzmin = min(j[2::3])

          for k in list_line_j: #test for intersection between each line making up plane i and the line of intersection to find the points of intersection
            try:
              point_intersection = line_of_intersection.intersect_line(k)
              if(point_intersection[0] > jxmin and point_intersection[0] < jxmax and point_intersection[1] > jymin and point_intersection[1] < jymax and point_intersection[2] > jzmin and point_intersection[2] < jzmax):
                #print(point_intersection)
                intersects_j.append(point_intersection.tolist())
            except:
              #print("b")
              pass
          if(len(intersects_j) < 2):
            breakcounter2 += 1
            continue

          print(intersects_i)
          xmax = max(intersects_i[0][0], intersects_i[1][0])
          xmin = min(intersects_i[0][0], intersects_i[1][0])
          ymax = max(intersects_i[0][1], intersects_i[1][1])
          ymin = min(intersects_i[0][1], intersects_i[1][1])
          zmax = max(intersects_i[0][2], intersects_i[1][2])
          zmin = min(intersects_i[0][2], intersects_i[1][2])

          for k in intersects_j:
            if(k[0] > xmin and k[0] < xmax and k[1] > ymin and k[1] < ymax and k[2] > zmin and k[2] < zmax):
                counter += 1
                break

      print(counter)

    def areas(self):
        #print(self.nodes)
        areas = np.zeros((self.N - 1, self.N - 1))
        for i in range(self.N - 1):
          for j in range(self.N - 1):
            p1 = i*self.N + j
            p2 = i*self.N + j + 1
            p3 = i*self.N + j + self.N
            p4 = i*self.N + j + 1 + self.N
            d1 = distance(self.nodes[p1], self.nodes[p2])
            d2 = distance(self.nodes[p1], self.nodes[p3])
            d3 = distance(self.nodes[p1], self.nodes[p4])
            d4 = distance(self.nodes[p2], self.nodes[p4])
            d5 = distance(self.nodes[p3], self.nodes[p4])
            areas[i][j] = heron(d1, d3, d4) + heron(d2, d3, d5)
        #print(areas)
        return areas

    def astar(self, maze, start, end):
        """Returns a list of tuples as a path from the given start to the given end in the given maze"""

        # Create start and end node
        start_node = Node(None, start)
        start_node.g = start_node.h = start_node.f = 0
        end_node = Node(None, end)
        end_node.g = end_node.h = end_node.f = 0

        # Initialize both open and closed list
        open_list = []
        closed_list = []

        # Add the start node
        open_list.append(start_node)

        # Loop until you find the end
        while len(open_list) > 0:

            # Get the current node
            current_node = open_list[0]
            current_index = 0
            for index, item in enumerate(open_list):
                if item.f < current_node.f:
                    current_node = item
                    current_index = index

            # Pop current off open list, add to closed list
            open_list.pop(current_index)
            closed_list.append(current_node)

            # Found the goal
            if current_node == end_node:
                path = []
                current = current_node
                while current is not None:
                    path.append(current.position)
                    current = current.parent
                return path[::-1] # Return reversed path

            # Generate children
            children = []
            for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares

                # Get node position
                node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

                # Make sure within range
                if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                    continue

                # Make sure walkable terrain
                if maze[node_position[0]][node_position[1]] != 0:
                    continue

                # Create new node
                new_node = Node(current_node, node_position)

                # Append
                children.append(new_node)

            # Loop through children
            for child in children:

                # Child is on the closed list
                for closed_child in closed_list:
                    if child == closed_child:
                        continue

                # Create the f, g, and h values
                child.g = current_node.g + np.linalg.norm(self.nodes[child.position[0]*self.N + child.position[1]] - self.nodes[end_node.position[0]*self.N + end_node.position[1]], ord=2)
                child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
                child.f = child.g + child.h

                # Child is already in the open list
                for open_node in open_list:
                    if child == open_node and child.g > open_node.g:
                        continue

                # Add the child to the open list
                open_list.append(child)

    def euclidian_distance_between_topographic_points(self, list_of_points):
        counter = 0
        first_point = list_of_points[0]
        for point in list_of_points[1:]:
          counter += np.linalg.norm(self.nodes[first_point[0]*self.N + first_point[1]] - self.nodes[point[0]*self.N + point[1]], ord=2)
          first_point = point
        return counter

    def euclidian_comparison(self):
        grid = [[0 for i in range(som.N)] for j in range(som.N)]
        distance_between_data_points = [] # walking distance between two people
        distance_from_data_point1_to_bmu = [] # walking distance to train station
        distance_from_data_point2_to_bmu = [] # walking distance to train station
        distance_between_bmus_diagonal = [] # drving distance between train stations
        distance_between_bmus_edges = [] # railway distance between train stations
        index_arr = np.random.randint(len(self.teachers),size=len(self.teachers)) #first num is # of data points, second num is # of iterations
        bmu1 = self._best_matching_unit(self.teachers[index_arr[0]])
        data_point1 = index_arr[0]
        for data_point2 in index_arr[1:]:
            bmu2 = self._best_matching_unit(self.teachers[data_point2])
            path = som.astar(grid, (bmu1[0], bmu2[1]), (bmu2[0], bmu2[1]))
            distance_between_data_points.append(np.linalg.norm(self.teachers[data_point1] - self.teachers[data_point2], ord=2))
            distance_from_data_point1_to_bmu.append(np.linalg.norm(self.teachers[data_point1] - self.nodes[bmu1[0]*self.N + bmu1[1]], ord=2))
            distance_from_data_point2_to_bmu.append(np.linalg.norm(self.teachers[data_point2] - self.nodes[bmu2[0]*self.N + bmu2[1]], ord=2))
            distance_between_bmus_diagonal.append(np.linalg.norm(self.nodes[bmu1[0]*self.N + bmu1[1]] - self.nodes[bmu2[0]*self.N + bmu2[1]], ord=2))
            distance_between_bmus_edges.append(som.euclidian_distance_between_topographic_points(path))
            bmu1 = bmu2
            data_point1 = data_point2
        information = [["Distance between data points:", ("Mean", statistics.mean(distance_between_data_points)), ("Median", statistics.median(distance_between_data_points)), ("Stdev", statistics.stdev(distance_between_data_points)), ("Range", max(distance_between_data_points) - min(distance_between_data_points))],
                       ["Distance between data point1 to bmu:", ("Mean", statistics.mean(distance_from_data_point1_to_bmu)), ("Median", statistics.median(distance_from_data_point1_to_bmu)), ("Stdev", statistics.stdev(distance_from_data_point1_to_bmu)), ("Range", max(distance_from_data_point1_to_bmu) - min(distance_from_data_point1_to_bmu))],
                       ["Distance between data point2 to bmu:", ("Mean", statistics.mean(distance_from_data_point2_to_bmu)), ("Median", statistics.median(distance_from_data_point2_to_bmu)), ("Stdev", statistics.stdev(distance_from_data_point2_to_bmu)), ("Range", max(distance_from_data_point2_to_bmu) - min(distance_from_data_point2_to_bmu))],
                       ["Distance between bmus diagonal:", ("Mean", statistics.mean(distance_between_bmus_diagonal)), ("Median", statistics.median(distance_between_bmus_diagonal)), ("Stdev", statistics.stdev(distance_between_bmus_diagonal)), ("Range", max(distance_between_bmus_diagonal) - min(distance_between_bmus_diagonal))],
                       ["Distance between bmus along edges:", ("Mean", statistics.mean(distance_between_bmus_edges)), ("Median", statistics.median(distance_between_bmus_edges)), ("Stdev", statistics.stdev(distance_between_bmus_edges)), ("Range", max(distance_between_bmus_edges) - min(distance_between_bmus_edges))]]
        return information

def heron(a, b, c):
    s = 0.5 * (a + b + c)
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))
    return area

def distance(a, b):
    sum = 0
    for i in range(len(a)):
        sum = sum + pow((a[i] - b[i]), 2)
    return pow(sum, 0.5)

def sklearn_iris():
    # Load data
    iris = load_iris(as_frame=True)
    # Create a dataframe
    df = iris['data'].join(iris['target'])

    random_d = df.values.tolist()
    for i in (random_d): #trying to delete any string in the dataset
      for j in range(len(i)-1, -1, -1):
        if type(i[j]) == str:
          i.remove(i[j])

    iris_som = SOM(dataset=df, initMethod="random", N=20, size=10000, seed=None)


# Initialize our SOM
som = SOM(dataset="swiss_roll", topoMap = "square", initMethod="random", N=10, size=10000, seed=None)
#som_to_compare = SOM(dataset="nocolor_point_cloud", topoMap = "square", initMethod="random", N=10, size=10000, seed=None) # 20x20 grid

# Initial map
som.plot_dataset()
som.plot_nodes()
plt.show()

# Training
#som.train()
som.train_log()
#indexList = [1,2,3,4,5,6,7,8,9,10]
#som.train_list(indexList)

# Final map
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
som.plot_dataset(fig, ax)
som.plot_nodes(fig, ax)
plt.show()

# Visualization methods
# som.edge_map()
# som.u_matrix()
som.button_plot()
#information = som.euclidian_comparison()
#for array in information:
#  print(array)


# Error calculations
#color_list = som.grader(5)
#nocolor_list = som_to_compare.grader(5)

#print("\nWith color init: ", color_list)
#print("Without color init: ", nocolor_list)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


MessageError: Error: credential propagation was unsuccessful